### Utilities

General purpose utilities including setting device, moving data and models to devices, setting seed

In [ ]:
#| default_exp utils

In [ ]:
#|export
import random
import numpy as np
import torch
from torch import nn

from torch.utils.data import default_collate
from typing import Mapping
import fastcore.test as fct

In [ ]:
#|export
def_device = 'mps' if torch.backends.mps.is_available() else 'cuda' if torch.cuda.is_available() else 'cpu'

def to_device(x, device=def_device):
    if isinstance(x, torch.Tensor): return x.to(device)
    if isinstance(x, Mapping): return {k:v.to(device) for k,v in x.items()}
    return type(x)(to_device(o, device) for o in x)

def collate_device(b): return to_device(default_collate(b))

In [ ]:
#|export
def to_cpu(x):
    """recursively move items to the cpu.  Works with tuples, lists or dictionaries 
    of tensors. As well as moving to the cpu detaches the tensor.  If the tensor is 16 bit then
    returns a standard float tensor
    """
    # Iterate through a dictionary
    if isinstance(x, Mapping): return {k:to_cpu(v) for k,v in x.items()}
    # Iteratively move all items in a list into a new list
    if isinstance(x, list): return [to_cpu(o) for o in x]
    # Convert a tuple by first converting to a list and then re-creating after moving
    if isinstance(x, tuple): return tuple(to_cpu(list(x)))
    try:
        res = x.detach().cpu()
    except: raise AttributeError("Attempting to convert item without detach method: {type(x)}")
    return res.float() if res.dtype==torch.float16 else res

Check that to_cpu works with dictionary of tensors

In [ ]:
test_dict = {"image":torch.tensor(2.5), "label": torch.tensor(1, dtype=torch.long)}
dict_on_dev = to_device(test_dict)
test_dict_cpu = to_cpu(dict_on_dev)
fct.all_equal(test_dict, test_dict_cpu)

True

Check that use of non tensors raises an assertion error

In [ ]:
with fct.ExceptionExpected():
    _ = to_cpu(5.6)

Check that to_cpu works with tensor

In [ ]:
test_tensor = (torch.tensor(5.6), torch.tensor(5, dtype=torch.long))
test_tensor_on_dev = to_device(test_tensor)
test_tensor_cpu = to_cpu(test_tensor_on_dev)
for x, y in zip(test_tensor, test_tensor_cpu):
    fct.equals(x,y)

In [ ]:
#|export
def set_seed(seed: int, deterministic:bool=False):
    """ Sets the seeds for torch, random and numpy.  If the deterministic flag is set torch will 
    attempt to use deterministic algorithms, if these are not available an error will be raised
    """
    torch.use_deterministic_algorithms(deterministic)
    torch.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)

## Test random number generation
Generate random number after seed, reapply seed, regenerate numbers and check equal

In [ ]:
set_seed(42)
rand_num_1 = random.randint(1, 1000)
torch_num_1 = torch.randint(1, 1000, (1,))
np_num_1 = np.random.randint(0, 1000, (1,))
set_seed(42)
rand_num_2 = random.randint(1, 1000)
torch_num_2 = torch.randint(1, 1000, (1,))
np_num_2 = np.random.randint(0, 1000, (1,))

fct.equals(rand_num_1, rand_num_2)
fct.equals(torch_num_1, torch_num_1)
fct.equals(np_num_1, np_num_2)

True

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()